Load Packages

In [10]:
import pickle
import os
import numpy as np
# from sklearn.model_selection import train_test_split
import gzip
import shutil
# import os
# import numpy as np
from sklearn.utils import shuffle
# Paths of datasets
full_game_save_path = 'C:/Users/jaspa/Grant ML/ranked_full_subfolders'
segment_save_path_subfolder = 'C:/Users/jaspa/Grant ML/ranked_segments_subfolders'
segment_save_path_bulk = 'C:/Users/jaspa/Grant ML/ranked_segments_bulk'

Create a training, test, and holdout set for a particular set of charcters

In [11]:
segment_save_path_subfolder = 'C:/Users/jaspa/Grant ML/ranked_segments_subfolders'
character_segment_counts = {'BOWSER': 9563, 'CAPTAIN_FALCON': 153832, 'DONKEY_KONG': 11970, 'DR_MARIO': 29235, 'FALCO': 424029, 'FOX': 489437, 'GAME_AND_WATCH': 14105, 'GANONDORF': 45758, 'ICE_CLIMBERS': 58698, 'JIGGLYPUFF': 138053, 'KIRBY': 3396, 'LINK': 21630, 'LUIGI': 39558, 'MARIO': 12368, 'MARTH': 249755, 'MEWTWO': 20922, 'NESS': 27592, 'PEACH': 144477, 'PICHU': 1726, 'PIKACHU': 30054, 'ROY': 9957, 'SAMUS': 85573, 'SHEIK': 189710, 'YOSHI': 41101, 'YOUNG_LINK': 12783, 'ZELDA': 3936}

selected_characters = [
    'FOX', 
    'FALCO', 
    'MARTH', 
    'SHEIK', 
    'CAPTAIN_FALCON', 
    'PEACH', 
    'JIGGLYPUFF', 
    'SAMUS', 
    # 'ICE_CLIMBERS', 
    # 'GANONDORF', 
    # 'YOSHI', 
    # 'LUIGI', 
    # 'PIKACHU', 
    # 'DR_MARIO', 
    # 'NESS', 
    # 'LINK', 
    # 'MEWTWO', 
    # 'GAME_AND_WATCH', 
    # 'DONKEY_KONG', 
    # 'YOUNG_LINK', 
    # 'MARIO', 
    # 'ROY', 
    # 'BOWSER', 
    # 'ZELDA', 
    # 'KIRBY', 
    # 'PICHU'
    ]

# Determine the minimum number of segments among the selected characters
min_segments = min([character_segment_counts[char] for char in selected_characters])
# min_segments = 2

# Map each selected character to an integer
char_to_int = {char: i for i, char in enumerate(selected_characters)}

# Initialize a dictionary to hold file paths organized by character
character_files = {char: [] for char in selected_characters}

# Collect all file paths organized by character
for character in selected_characters:
    char_path = os.path.join(segment_save_path_subfolder, character)
    for opponent in os.listdir(char_path):
        opponent_path = os.path.join(char_path, opponent)
        files = [os.path.join(opponent_path, file) for file in os.listdir(opponent_path)]
        character_files[character].extend(files)

# Sample min_segments for each selected character
X = []
y = []
for character, files in character_files.items():
    sampled_files = np.random.choice(files, min_segments, replace=False)
    X.extend(sampled_files)
    y.extend([char_to_int[character]] * min_segments)

# Shuffle the dataset to mix up the order of characters
X, y = shuffle(np.array(X), np.array(y), random_state=42)

# At this point, X and y are your balanced dataset ready for further processing
print('number of data points: ',X.shape[0])

number of data points:  769160


In [13]:

def save_data_with_mmap(X, y, save_path):
    """
    Saves file paths and labels using numpy.memmap for efficient memory usage.

    Args:
        X (numpy.array): Array of file paths.
        y (numpy.array): Array of labels.
        save_path (str): Directory where the data should be saved.
    """
    # Ensure save_path exists
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # Define the paths for the memory-mapped files
    file_paths_mmap = os.path.join(save_path, 'ranked_file_paths.dat')
    labels_mmap = os.path.join(save_path, 'ranked_label_list.dat')

    # Create and save the memory-mapped arrays
    # Mode 'w+' creates a file for reading and writing, initializing the file to the given shape and dtype
    X_mmap = np.memmap(file_paths_mmap, dtype=np.str_, mode='w+', shape=X.shape)
    X_mmap[:] = X
    del X_mmap  # Flush the changes

    y_mmap = np.memmap(labels_mmap, dtype=np.int16, mode='w+', shape=y.shape)
    y_mmap[:] = y
    del y_mmap  # Flush the changes
    
save_data_with_mmap(X,y,'C:/Users/jaspa/Grant ML/slp/data/' )


ValueError: cannot mmap an empty file

In [14]:
print(X.shape)
print(y.shape)

(769160,)
(769160,)


In [ ]:
def array_memory_size(array):
    """
    Calculate the memory size of a NumPy array.

    Args:
        array (numpy.array): The NumPy array for which to calculate memory usage.

    Returns:
        int: Memory usage of the array in bytes.
    """
    # Get the size of one array element in bytes
    element_size = array.dtype.itemsize
    # Get the total number of elements in the array
    total_elements = array.size
    # Calculate total memory usage
    memory_usage = element_size * total_elements/ (1024**3)
    return memory_usage

print(array_memory_size(X))

0.39828285574913025


In [ ]:
save_path = 'C:/Users/jaspa/Grant ML/slp/data/'


with gzip.open(os.path.join(save_path,'ranked_file_paths.npy.gz'), 'wb') as f:
    np.save(f, X)

with gzip.open(os.path.join(save_path,'ranked_label_list.npy.gz'), 'wb') as f:
    np.save(f, y)

